docker build -t hate-speech-detection-tf-serving . <br>
docker run -p 8080:8501 hate-speech-detection-tf-serving

In [1]:
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [2]:
df = pd.read_csv('data\data.csv')
df.head()

,text,label
0,It should be legal for a husband to rape his wife,1
1,dalits are lowlife scum,1
2,dalits are lowlives,1
3,It was a better world when women dare not ques...,1
4,I don't work this hard so that those immigrant...,1


In [3]:
data_test = df['text'][0]
data_test

'It should be legal for a husband to rape his wife'

In [4]:
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/hate-speech-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1712553083'}]}


In [5]:
def encode_text(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    return base64.b64encode(example).decode()

def prepare_data(text):
    encoded_example = encode_text(text)
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [{
            "examples": {"b64": encoded_example}
        }]
    })

json_data = prepare_data(data_test)

endpoint = "http://localhost:8080/v1/models/hate-speech-detection-model:predict"
response = requests.post(endpoint, data=json_data)

predictions = response.json().get("predictions", [])
result = "Hate Speech" if predictions and predictions[0][0] > 0.6 else "Not Hate Speech" if predictions else "Error: No predictions found."

print(result)

Hate Speech
